In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [3]:
#Load data
df = pd.read_csv('/content/DECENNIALDPMP2020.DP3-2023-04-13T182922.csv')
df

,Label (Grouping),Commonwealth of the Northern Mariana Islands!!Number,Commonwealth of the Northern Mariana Islands!!Percent
0,EMPLOYMENT STATUS,NaN,NaN
1,Population 16 years and over,"35,396",100.0%
2,In labor force,"21,843",61.7%
3,Civilian labor force,"21,827",61.7%
4,Employed,"18,759",53.0%
...,...,...,...
164,Related children of the householde...,"6,522",48.5%
165,18 years and over,"11,291",33.7%
166,18 to 64 years,"10,569",34.4%
167,65 years and over,722,25.9%


In [4]:
# Checking a datas type missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 3 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   Label (Grouping)                                       169 non-null    object
 1   Commonwealth of the Northern Mariana Islands!!Number   159 non-null    object
 2   Commonwealth of the Northern Mariana Islands!!Percent  159 non-null    object
dtypes: object(3)
memory usage: 4.1+ KB


In [5]:
# Check class duplicates
df['Commonwealth of the Northern Mariana Islands!!Number'].value_counts()

18,759    4
21,827    2
16        2
14,282    2
17,687    2
         ..
13,509    1
4,821     1
429       1
47,128    1
4,383     1
Name: Commonwealth of the Northern Mariana Islands!!Number, Length: 151, dtype: int64

In [6]:
# check dupl;iates
df.duplicated().sum()

4

In [7]:
# drop duplocates
df= df.drop_duplicates()
df.duplicated().sum()

0

In [9]:
# quick check for outlierts
df.describe()

,Label (Grouping),Commonwealth of the Northern Mariana Islands!!Number,Commonwealth of the Northern Mariana Islands!!Percent
count,165,155,155
unique,130,151,109
top,With related children under 5 years,331,(X)
freq,4,2,16


In [10]:
# prepare train and test set for modeling 
X = df.drop(columns = ['Commonwealth of the Northern Mariana Islands!!Percent'])
y = df['Commonwealth of the Northern Mariana Islands!!Percent']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.head()

,Label (Grouping),Commonwealth of the Northern Mariana Islands!!Number
121,Mean retirement income (dollars),"23,248"
96,Households,"14,282"
137,"$125,000 or more",537
37,Bicycle,162
85,Unemployed,"2,777"


In [12]:
# check catefories

for col in df.select_dtypes('object'):
  print(df[col].value_counts())

            With related children under 5 years                         4
        With related children under 18 years                            4
        In labor force                                                  3
        Not in labor force                                              3
        $30,000 to $39,999                                              2
                                                                       ..
        Agriculture, forestry, fishing and hunting, and mining          1
INDUSTRY [2]                                                            1
        Production, transportation, and material moving occupations     1
        Natural resources, construction, and maintenance occupations    1
        Unrelated individuals 15 years and over                         1
Name: Label (Grouping), Length: 130, dtype: int64
331       2
21,827    2
18,759    2
16        2
23,248    1
         ..
2,259     1
13,509    1
4,821     1
429       1
4,383     1
Name

In [13]:
# Seems to represent missing data

df = df.replace('?', np.nan)

In [14]:
# class balance
y_train.value_counts(normalize=True)

100.0%    0.105263
(X)       0.096491
53.0%     0.026316
0.3%      0.017544
0.1%      0.017544
            ...   
10.6%     0.008772
48.5%     0.008772
32.4%     0.008772
2.9%      0.008772
9.4%      0.008772
Name: Commonwealth of the Northern Mariana Islands!!Percent, Length: 85, dtype: float64

In [16]:
# preproceessing 

cat_selector = make_column_selector(dtype_include = 'Object')
imputer = SimpleImputer(strategy='most_frequent')

In [17]:
from pandas.core.array_algos import transforms
# one hot encoder
ohe = OneHotEncoder(handle_unknown='ignore')
cat_pipe = make_pipeline(imputer, ohe)
cat_tuple = (cat_pipe, cat_selector)


transforms= make_column_transformer(cat_tuple, remainder='passthrough')